# 📊 Cleaning Data from Numbeo for countries in Southeast Asia 

I collected data from Numbeo about living costs in Southeast Asia countries using web-scraping with BeautifulSoup in Python. I saved this in a CSV file. Now, in this notebook, I'll clean this data, making it ready for my main project.

## What's the Project About?

I'm working on a big project where I want to know more about:

1. **How much it costs to live in different countries from Southeast Asia:** This is the data from Numbeo that tells us about things like food, rent and other costs in various cities.
2. **Visa rules:** This is about what rules people need to follow to visit or live in Southeast countries.
3. **Safety and health:** This will tell us how safe a place is and a map with details about where to find hospitals and how to stay safe.
4. **Weather:** This will help us understand what the climate is like in different parts of these countries.

**What I Want at the End:** After collecting and cleaning all this data, I want to create a web platform that's like a toolkit for digital nomads heading to Southeast Asia.

Now, let's start cleaning the data from Numbeo!


## Importing Necessary Libraries

Before we start with data cleaning, we need to import the required Python libraries. 🐍

In [2]:
import pandas as pd
import numpy as np

## Loading the Data

* We will load the data from the CSV file into a Pandas DataFrame for easy manipulation.

In [3]:
df = pd.read_csv("C:\\Users\\LEGION\\Desktop\\Nomads in Asia\\WS_Numbeo\\numbeo_data_usd.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       715 non-null    object
 1   1       715 non-null    object
 2   2       715 non-null    object
 3   3       582 non-null    object
dtypes: object(4)
memory usage: 22.5+ KB


## Preview the Data

* Let's take a quick look at the top rows of our data.
* I renamed the columns for clarity.

In [7]:
df.head(10)
df.columns = ['Country', 'Expenses', 'Price', 'Range']
df

,Country,Expenses,Price,Range
0,Singapore,Restaurants,Edit,Range
1,Singapore,"Meal, Inexpensive Restaurant",9.53 $,4.40-20.17
2,Singapore,"Meal for 2 People, Mid-range Restaurant, Three...",73.34 $,44.00-128.34
3,Singapore,McMeal at McDonalds (or Equivalent Combo Meal),7.33 $,5.87-8.80
4,Singapore,Domestic Beer (0.5 liter draught),7.33 $,3.67-11.00
...,...,...,...,...
710,Timor-Leste,Price per Square Meter to Buy Apartment in Cit...,"4,666.67 $","4,000.00-5,000.00"
711,Timor-Leste,Price per Square Meter to Buy Apartment Outsid...,"4,000.00 $",NaN
712,Timor-Leste,Salaries And Financing,Edit,NaN
713,Timor-Leste,Average Monthly Net Salary (After Tax),"5,121.71 $",NaN


## Data Cleaning:

* In the **'Price'** column, I replaced any **'Edit'** or **'?'** values with blanks (**NaN**). Then, I removed dollar signs and commas from the prices. Finally, I changed the text prices into actual number values for easier calculations.


In [7]:
# Replace 'Edit' and '?' with NaN
df['Price'].replace(['Edit', '?'], np.nan, inplace=True)

# Ensure all remaining values are strings before applying further replacements
df['Price'] = df['Price'].astype(str)

# Remove the dollar sign and strip white spaces
df['Price'] = df['Price'].str.replace('$', '').str.strip()

# Remove commas and convert to float
df['Price'] = df['Price'].str.replace(',', '').astype(float)

* In the **'Range'** column, I took out any commas. Next, I split this column in two: '**Min Range**' and '**Max Range**', based on the dash in between values. Lastly, I made sure both these new columns had number values for easier calculations.

In [8]:
# Remove commas from the 'Range' column
df['Range'] = df['Range'].str.replace(',', '')

# Split Range into Min and Max columns and convert to float
df[['Min Range', 'Max Range']] = df['Range'].str.split('-', expand=True)

# Convert the new Min Range and Max Range columns to numeric type
df['Min Range'] = pd.to_numeric(df['Min Range'], errors='coerce')
df['Max Range'] = pd.to_numeric(df['Max Range'], errors='coerce')


* I created a mapping to categorize individual expenses into their respective categories for better clarity and to keep the data structured. I keep this data in **Category** column.

In [9]:
expense_to_category = {
    
    #Mappings for "Restaurant" category:
    "Meal, Inexpensive Restaurant": "Restaurant",
    "Meal for 2 People, Mid-range Restaurant, Three-course": "Restaurant",
    "McMeal at McDonalds (or Equivalent Combo Meal)": "Restaurant",
    "Domestic Beer (0.5 liter draught)": "Restaurant",
    "Imported Beer (0.33 liter bottle)": "Restaurant",
    "Cappuccino (regular)": "Restaurant",
    "Coke/Pepsi (0.33 liter bottle)": "Restaurant",
    "Water (0.33 liter bottle)": "Restaurant",
    
    #Mappings for "Markets" category:
    "Milk (regular), (1 liter)": "Markets",
    "Loaf of Fresh White Bread (500g)": "Markets",
    "Rice (white), (1kg)": "Markets",
    "Eggs (regular) (12)": "Markets",
    "Local Cheese (1kg)": "Markets",
    "Chicken Fillets (1kg)": "Markets",
    "Beef Round (1kg) (or Equivalent Back Leg Red Meat)": "Markets",
    "Apples (1kg)": "Markets",
    "Banana (1kg)": "Markets",
    "Oranges (1kg)": "Markets",
    "Tomato (1kg)": "Markets",
    "Potato (1kg)": "Markets",
    "Onion (1kg)": "Markets",
    "Lettuce (1 head)": "Markets",
    "Water (1.5 liter bottle)": "Markets",
    "Bottle of Wine (Mid-Range)": "Markets",
    "Domestic Beer (0.5 liter bottle)": "Markets",
    "Imported Beer (0.33 liter bottle)": "Markets",
    "Cigarettes 20 Pack (Marlboro)": "Markets",
    
    #Mappings for "Transportation" category:
    "One-way Ticket (Local Transport)": "Transportation",
    "Monthly Pass (Regular Price)": "Transportation",
    "Taxi Start (Normal Tariff)": "Transportation",
    "Taxi 1km (Normal Tariff)": "Transportation",
    "Taxi 1hour Waiting (Normal Tariff)": "Transportation",
    "Gasoline (1 liter)": "Transportation",
    "Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car)": "Transportation",
    "Toyota Corolla Sedan 1.6l 97kW Comfort (Or Equivalent New Car)": "Transportation",
    
    #Mappings for "Utilities (Monthly)" category:
    "Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment": "Utilities",
    "Mobile Phone Monthly Plan with Calls and 10GB+ Data": "Utilities",
    "Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)": "Utilities",
    
    #Mappings for "Sports And Leisure" category:
    "Fitness Club, Monthly Fee for 1 Adult": "Sports And Leisure",
    "Tennis Court Rent (1 Hour on Weekend)": "Sports And Leisure",
    "Cinema, International Release, 1 Seat": "Sports And Leisure",
    
    #Mappings for "Childcare" category:
    "Preschool (or Kindergarten), Full Day, Private, Monthly for 1 Child": "Childcare",
    "International Primary School, Yearly for 1 Child": "Childcare",
    
    #Mappings for "Clothing And Shoes" category:
    "1 Pair of Jeans (Levis 501 Or Similar)": "Clothing And Shoes",
    "1 Summer Dress in a Chain Store (Zara, H&M, ...)": "Clothing And Shoes",
    "1 Pair of Nike Running Shoes (Mid-Range)": "Clothing And Shoes",
    "1 Pair of Men Leather Business Shoes": "Clothing And Shoes",
    
    #Mappings for "Rent Per Month" category:
    "Apartment (1 bedroom) in City Centre": "Rent Per Month",
    "Apartment (1 bedroom) Outside of Centre": "Rent Per Month",
    "Apartment (3 bedrooms) in City Centre": "Rent Per Month",
    "Apartment (3 bedrooms) Outside of Centre": "Rent Per Month",
    
    #Mappings for "Buy Apartment Price" category:
    "Price per Square Meter to Buy Apartment in City Centre": "Buy Apartment Price",
    "Price per Square Meter to Buy Apartment Outside of Centre": "Buy Apartment Price",
    
    #Mappings for "Salaries And Financing" category:
    "Average Monthly Net Salary (After Tax)": "Salaries And Financing",
    "Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate": "Salaries And Financing"
}
# Insert 'Category' column at the 3nd position (index 2)
df.insert(2, 'Category', df['Expenses'].map(expense_to_category))
df.head(5)

,Country,Expenses,Category,Price,Range,Min Range,Max Range
0,Singapore,Restaurants,NaN,NaN,Range,NaN,NaN
1,Singapore,"Meal, Inexpensive Restaurant",Restaurant,9.53,4.40-20.17,4.40,20.17
2,Singapore,"Meal for 2 People, Mid-range Restaurant, Three...",Restaurant,73.34,44.00-128.34,44.00,128.34
3,Singapore,McMeal at McDonalds (or Equivalent Combo Meal),Restaurant,7.33,5.87-8.80,5.87,8.80
4,Singapore,Domestic Beer (0.5 liter draught),Restaurant,7.33,3.67-11.00,3.67,11.00


* I deleted the rows containing information about the category because, previously I created a new column that provides this information.

In [10]:
# List of values to be excluded
exclude_values = ["Restaurants", "Markets", "Transportation", "Utilities (Monthly)", "Sports And Leisure", "Childcare", "Clothing And Shoes", "Rent Per Month", "Buy Apartment Price", "Salaries And Financing"]

# Exclude rows containing specific values in 'Expenses'
df = df[~df['Expenses'].isin(exclude_values)]

* I notice that we are missing some values for the **Category**, **Price**, **Range**, **Min Range**, **Max Range** columns. I am inspecting them and I will try to correct them where possible.<br><br>

* I will start with the **Category** column:

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 605 entries, 1 to 714
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    605 non-null    object 
 1   Expenses   605 non-null    object 
 2   Category   600 non-null    object 
 3   Price      603 non-null    float64
 4   Range      571 non-null    object 
 5   Min Range  571 non-null    float64
 6   Max Range  571 non-null    float64
dtypes: float64(3), object(4)
memory usage: 37.8+ KB


In [12]:
# Filter and display rows where 'Category' is NaN
nan_price_rows = df[df['Category'].isna()]
nan_price_rows

,Country,Expenses,Category,Price,Range,Min Range,Max Range
589,Brunei,Domestic Non-Alcoholic Beer (0.5 liter draught),NaN,1.83,1.47-3.67,1.47,3.67
590,Brunei,Imported Non-Alcoholic Beer (0.33 liter bottle),NaN,2.20,1.83-2.93,1.83,2.93
610,Brunei,Bottle of Non-Alcoholic Wine (Mid-Range),NaN,11.00,2.20-25.67,2.20,25.67
611,Brunei,Domestic Non-Alcoholic Beer (0.5 liter bottle),NaN,3.91,0.73-7.33,0.73,7.33
612,Brunei,Imported Non-Alcoholic Beer (0.33 liter bottle),NaN,1.25,1.03-1.47,1.03,1.47


* After inspecting these values on the website, I found what I need to add.

In [13]:
# Replace the missing values
df.loc[[589, 590], 'Category'] = "Restaurant"
df.loc[[610, 611, 612], 'Category'] = "Markets"

* Category done! Next, Price:

In [14]:
# Filter and display rows where 'Price' is NaN
nan_price_rows = df[df['Price'].isna()]
nan_price_rows

,Country,Expenses,Category,Price,Range,Min Range,Max Range
561,Laos,"Internet (60 Mbps or More, Unlimited Data, Cab...",Utilities,NaN,NaN,NaN,NaN
690,Timor-Leste,Mobile Phone Monthly Plan with Calls and 10GB+...,Utilities,NaN,NaN,NaN,NaN


* After conducting research, I extracted the price for "**Mobile Phone Monthly Plan with Calls and 10GB+ Data**" in **Timor-Leste** from (https://esims.io/countries/laos), a provider of prepaid eSIM plans for tourists. This data was accessed on 10/13/2023.


In [15]:
df.loc[(df['Country'] == 'Timor-Leste') & 
 (df['Expenses'] == 'Mobile Phone Monthly Plan with Calls and 10GB+ Data'), 'Price'] = 44

* The cost of "**Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)**" in **Laos** is currently not available. Efforts to find this data from various sources were unsuccessful. The analysis will proceed with this data point as **NaN** and interpretations will be made accordingly.


### Note on Missing Values in "Range", "Min Range", and "Max Range" Columns

Entries with NaN in the "Range", "Min Range", and "Max Range" columns indicate that the data for those specific ranges is not available. Despite efforts to find substitute values, no suitable data was found. In this case, statistical imputations like using medians or means are not applicable due to the nature and specificity of the data.


In [16]:
# Filter and display rows where 'Range' is NaN
nan_price_rows = df[df['Range'].isna()]
nan_price_rows

,Country,Expenses,Category,Price,Range,Min Range,Max Range
63,Singapore,Average Monthly Net Salary (After Tax),Salaries And Financing,4874.45,NaN,NaN,NaN
128,Malaysia,Average Monthly Net Salary (After Tax),Salaries And Financing,793.27,NaN,NaN,NaN
193,Indonesia,Average Monthly Net Salary (After Tax),Salaries And Financing,324.74,NaN,NaN,NaN
258,Philippines,Average Monthly Net Salary (After Tax),Salaries And Financing,339.91,NaN,NaN,NaN
323,Thailand,Average Monthly Net Salary (After Tax),Salaries And Financing,532.21,NaN,NaN,NaN
388,Vietnam,Average Monthly Net Salary (After Tax),Salaries And Financing,441.15,NaN,NaN,NaN
426,Myanmar,Volkswagen Golf 1.4 90 KW Trendline (Or Equiva...,Transportation,35000.00,NaN,NaN,NaN
453,Myanmar,Average Monthly Net Salary (After Tax),Salaries And Financing,309.15,NaN,NaN,NaN
518,Cambodia,Average Monthly Net Salary (After Tax),Salaries And Financing,293.50,NaN,NaN,NaN
540,Laos,Tomato (1kg),Markets,4.41,NaN,NaN,NaN
